# ProgressBar

如何使用 tqdm 生成进度条，参考下面的补丁：<https://gitee.com/zhyantao/misc/commit/4305f7a3fe68357d5acb34e5183a437b17dee3ae>


```diff
diff --git a/scripts/extract_persons.py b/scripts/extract_persons.py
index dd787af2dcb04f332defd52554119248706c004a..adf7d6ff64d37d9b098d217f6d00133b83f2a20f 100644
--- a/scripts/extract_persons.py
+++ b/scripts/extract_persons.py
@@ -2,8 +2,10 @@ import cv2
 import numpy as np
 import os
 import requests
+from tqdm import tqdm
 
 yolo_files_dir = "yolo_files"  # YOLO 文件存放目录
+video_dir = f"{yolo_files_dir}/videos"  # 视频文件存放目录
 
 
 def download_file(url, save_path):
@@ -100,9 +102,15 @@ def detect_objects(frame, net, output_layers, classes):
     return boxes, confidences, class_ids, indexes
 
 
-def extract_persons(
-    video_path, output_folder="extracted_persons", frame_interval=300
-):
+def format_progress_bar(progress, bar_length=100):
+    """格式化进度条为 |########         | 94% 的形式"""
+    filled_length = int(round(bar_length * progress))
+    bar = "#" * filled_length + " " * (bar_length - filled_length)
+    percent = round(progress * 100, 2)
+    return f"|{bar}| {percent}%"
+
+
+def extract_persons(video_path, output_folder="extracted_persons", frame_interval=300):
     # 创建输出文件夹
     if not os.path.exists(output_folder):
         os.makedirs(output_folder)
@@ -176,6 +184,11 @@ def extract_persons(
 
         frame_count += 1
 
+        # 使用 tqdm 显示处理进度
+        progress = frame_count / total_frames
+        progress_bar = format_progress_bar(progress)
+        print(f"\r处理进度: {progress_bar}", end="", flush=True)
+
     cap.release()
     print(f"提取完成！共提取到 {saved_frame_count} 张人像图片。")
 
@@ -190,7 +203,6 @@ if __name__ == "__main__":
     download_yolo_files()
 
     # 将需要检测的视频文件放在 videos 目录下
-    video_dir = f"{yolo_files_dir}/videos"
     os.makedirs(video_dir, exist_ok=True)
 
     # 枚举 video_dir 下的 mp4 文件
```